# McStas to NeXus for ESTIA

This notebook converts data from a McStas simulation output (`.h5` filetype) to a NeXus file that uses a file for the Estia instrument (written by CODA) as a template for the geometry information.

It adds

* events to the `multiblade_detector` detector,
* a single value to the `proton_current` log,
* a single detector rotation value to the `detector_rotation` log,
* a single sample rotation value to the `sample_rotation` log.

In [ ]:
import numpy as np
import scipp as sc
import h5py
import shutil

from ess.estia.mcstas import parse_events_h5
from ess.estia.load import load_mcstas

In [ ]:
def replace_dataset(entry, name, values):
    attrs = dict(entry[name].attrs)
    del entry[name]
    dset = entry.create_dataset(name, data=values)
    dset.attrs.update(attrs)


def mcstas_to_nexus(
    mcstas_data_file: str,
    template_nexus_file: str,
    outfile: str,
    nevents_per_weight: float = 1,
):
    """
    Store the events from a McStas Estia simulation in a NeXus CODA file.

    Parameters
    ----------
    mcstas_data_file:
        Data file containing simulated McStas events.
    template_nexus_file:
        NeXus file containing geometry and instrument info, used as a template.
    outfile:
        Output file to be written.
    nevents_per_weight:
        Number of events to have in the output file
        per unit weight in the McStas data.
    """
    detector_entry_path = '/entry/instrument/multiblade_detector'

    with h5py.File(template_nexus_file, "r") as f:
        det_numbers = f[f"{detector_entry_path}/detector_number"][()]

    with h5py.File(mcstas_data_file) as f:
        events = parse_events_h5(f,  events_to_sample_per_unit_weight=nevents_per_weight)
    binned = load_mcstas(events).transpose(['strip', 'blade', 'wire', ])

    toa = (binned.bins.coords["event_time_zero"] + binned.bins.coords["event_time_offset"]).bins.concat().value

    det_numbers_matching_order_of_binned = det_numbers.reshape(binned.shape)
    det_numbers_matching_order_of_binned = det_numbers_matching_order_of_binned[::-1, ::-1, ::-1]
    # IMPORTANT! we need to sort the arrays below according to toa,
    # so that the event_index does not get messed up!
    event_id = sc.sort(
        (
            sc.bins_like(binned, sc.array(dims=binned.dims, values=det_numbers_matching_order_of_binned))
            .bins.concat()
            .value
        ),
        key=toa,
    )
    event_time_zero = sc.sort(binned.bins.coords["event_time_zero"].bins.concat().value, key=toa)
    event_time_offset = sc.sort(binned.bins.coords["event_time_offset"].bins.concat().value, key=toa)

    event_index = sc.DataArray(
        data=sc.ones_like(event_time_offset),
        coords={"event_time_zero": event_time_zero},
    ).group("event_time_zero")

    event_index = sc.cumsum(event_index.bins.size())
    event_index.values = np.concatenate([[0], event_index.values[:-1]])

    shutil.copyfile(template_nexus_file, outfile)

    with h5py.File(outfile, "r+") as f:

        # TODO: remove this when the files is fixed
        # Fix error in Nexus file
        f['/entry/instrument/source/transformations/location'][()] = -35.0512

        detector_rotation = f[f"{detector_entry_path}/transformations/detector_rotation"]
        replace_dataset(
            detector_rotation,
            name="value",
            values=[
                binned
                    .coords['detector_rotation']
                    .to(unit=detector_rotation['value'].attrs['units'])
                    .value
            ]
        )
        replace_dataset(
            detector_rotation,
            name="time",
            values=[
                event_time_zero
                    .min()
                    .to(unit=detector_rotation['time'].attrs['units'])
                    .value
                    .astype('uint64')
            ]
        )

        sample = f["entry/sample"]
        sample_rotation = sample.create_group('sample_rotation')
        sample_rotation.attrs['NX_class'] = 'NXlog'
        sample_rotation.create_dataset(
            'value',
            data=[
                binned
                .coords['sample_rotation']
                .to(unit='deg')
                .value
                .astype('float64')
            ],
        )
        sample_rotation['value'].attrs['units'] = 'degrees'
        sample_rotation.create_dataset(
            'time',
            data=[
                event_time_zero
                    .min()
                    .to(unit='ns')
                    .value
                    .astype('uint64')
            ],
        )
        sample_rotation['time'].attrs['units'] = 'ns'

        f["entry/neutron_prod_info/pulse_charge/value"][:] = 1.

        event_data = f[f"{detector_entry_path}/event_data"]
        replace_dataset(event_data, name="event_id", values=event_id.values.astype('uint32'))
        replace_dataset(
            event_data,
            name="event_time_offset",
            values=event_time_offset.to(
                unit=event_data["event_time_offset"].attrs["units"], copy=False
            ).values.astype('uint32'),
        )
        replace_dataset(event_data, name="event_index", values=event_index.values.astype('uint64'))
        replace_dataset(
            event_data,
            name="event_time_zero",
            values=event_index.coords["event_time_zero"]
            .to(unit=event_data["event_time_zero"].attrs["units"], copy=False)
            .values.astype('uint64'),
        )

### Generate nexus files from a Nexus template and McStas events

In [ ]:
from ess.estia.data import estia_mcstas_example


mcstas_to_nexus(
    mcstas_data_file=estia_mcstas_example('reference'),
    template_nexus_file="/home/johannes/Downloads/estia_999999_00008786.hdf",
    outfile='reference.nx',
    nevents_per_weight=1,
)

for i, example in enumerate(estia_mcstas_example('Ni/Ti-multilayer')):
    mcstas_to_nexus(
        mcstas_data_file=example,
        template_nexus_file="/home/johannes/Downloads/estia_999999_00008786.hdf",
        outfile=f'niti_ml_{i}.nx',
        nevents_per_weight=1,
    )